In [1]:
import numpy as np
from numpy import genfromtxt

In [2]:
def sigmoid(x):
    return 1/(1+np.exp(-x))

def softmax(x):
    x = x
    e_x = np.exp(x-np.max(x))
    return e_x / e_x.sum()

def test_loss(x_in, y_in, weights1, weights2, biases1, biases2):
        out2 = pred_out(x_in, y_in, weights1, weights2, biases1, biases2)
        loss = []
        for iter in range(out2.shape[1]):
            loss.append(MSE(out2[:, iter], y_in[:, iter]))
        return np.mean(loss)

def pred_out(x_in, y_in, weights1, weights2, biases1, biases2):
        out1_temp = np.matmul(weights1.T, x_in[:,:])+biases1
        out1 = sigmoid(out1_temp)
        out2_temp = np.matmul(weights2.T, out1)+biases2
        out2 = (out2_temp)
        return out2

def acc_out(x_in, y_in, weights1, weights2, biases1, biases2):
    pred = pred_out(x_in, y_in, weights1, weights2, biases1, biases2)
    sum_acc = 0
    for iter in range(pred.shape[1]):
#         print(iter)
        sum_acc+=(np.argmax(pred[:, iter])==np.argmax(y_in[:, iter]))
    return sum_acc/y_in.shape[1]

def MSE(yHat, y):
    return np.sum(np.multiply(yHat - y, yHat - y)) / y.size

def cross_entropy(predictions, targets, epsilon=1e-5):
    predictions = np.clip(predictions, epsilon, 1. - epsilon)
    ce = - np.mean(np.multiply(np.log(predictions),targets)) 
    return ce

In [3]:
##Load and segregate the data
letters = genfromtxt('energy_eff2.csv', delimiter=',')
# np.random.shuffle(letters)
print(letters.shape)
x_in = letters[:, 0:8]
x_mean = np.mean(x_in, axis = 0)
x_std = np.std(x_in, axis = 0)
y_in = letters[:, 8:10]
y_mean = np.mean(y_in, axis=0)
y_std = np.std(y_in, axis = 0)
# y_in = (y_in-y_mean)/y_std
x_in = (x_in-x_mean)/x_std
# print(np.std(y_in, axis=0))

(768, 10)


In [4]:
x_train = x_in[0:int(x_in.shape[0]*0.7), :]
y_train = y_in[0:int(x_in.shape[0]*0.7), :]

x_test = x_in[int(x_in.shape[0]*0.7):x_in.shape[0], :]
y_test = y_in[int(x_in.shape[0]*0.7):x_in.shape[0], :]

In [ ]:
hidden_size = 10
learning_rate = 0.005
num_epochs = 1000
alpha = 0.9
epsilon = 1e-12

weights1 = np.random.normal(size=(x_in.shape[1], hidden_size))
weights2 = np.random.normal(size=(hidden_size, y_in.shape[1]))
print(weights1.shape)
print(weights2.shape)
biases1 = np.random.normal(size=(hidden_size, 1))
biases2 = np.random.normal(size=(y_in.shape[1], 1))

x_train = np.matrix(x_train)
x_train = x_train.T
y_train = np.matrix(y_train)
y_train = y_train.T

x_test = np.matrix(x_test)
x_test = x_test.T
y_test = np.matrix(y_test)
y_test = y_test.T

(8, 10)
(10, 2)


In [ ]:
accuracies_train = []
accuracies_test = []
losses = []
test_losses = []

biases1_squares = 0
biases2_squares = 0
weights1_squares = 0
weights2_squares = 0

for epoch_num in range(num_epochs):
    if epoch_num%5==1:
        print("test_loss : ", test_losses[-1])
        print("train_loss : ", losses[-1])
        print("epoch_num : ", epoch_num)
    for sample_num in range(x_train.shape[1]):
#         print("sample_num : ", sample_num)
        #Forward propagation
        out1_temp = np.matmul(weights1.T, x_train[:,sample_num])+biases1
        out1 = sigmoid(out1_temp)
        out2_temp = np.matmul(weights2.T, out1)+biases2
        out2 = (out2_temp)
        #Back-propogation
        error_out = out2-y_train[:, sample_num]
        err_2 = np.matrix(error_out)
        grad_2 = np.matmul(out1, err_2.T)
        err_1 = np.multiply(np.matmul(weights2, err_2), np.multiply(out1, np.ones(out1.shape)-out1))
        grad_1 = np.multiply(x_train[:, sample_num], err_1.T)
        #Updating the weights
        
        biases1_squares = alpha*biases1_squares+(1-alpha)*np.multiply(err_1, err_1)
        biases2_squares = alpha*biases2_squares+(1-alpha)*np.multiply(err_2, err_2)
        weights1_squares = alpha*weights1_squares+(1-alpha)*np.multiply(grad_1, grad_1)
        weights2_squares = alpha*weights2_squares+(1-alpha)*np.multiply(grad_2, grad_2)
        
        biases1 = biases1 - np.multiply(learning_rate/(np.sqrt(biases1_squares)+epsilon), err_1)
        biases2 = biases2 - np.multiply(learning_rate/(np.sqrt(biases2_squares)+epsilon), err_2)
        weights1 = weights1 - np.multiply(learning_rate/(np.sqrt(weights1_squares)+epsilon), grad_1)
        weights2 = weights2 - np.multiply(learning_rate/(np.sqrt(weights2_squares)+epsilon), grad_2)
        #Updating the accuracy and loss
        #End of for loop
#     print("accuracy : ", np.mean(acc_arr))
    test_losses.append(test_loss(x_test, y_test, weights1, weights2, biases1, biases2))
#     accuracies_test.append(acc_out(x_test, y_test, weights1, weights2, biases1, biases2))
#     accuracies_train.append(acc_out(x_train, y_train, weights1, weights2, biases1, biases2))
    losses.append(test_loss(x_train, y_train, weights1, weights2, biases1, biases2))
    

test_loss :  166.81684165353306
train_loss :  185.3848234521781
epoch_num :  1
test_loss :  8.640668477172644
train_loss :  9.192988811989629
epoch_num :  6
test_loss :  7.173117209617851
train_loss :  7.162389256271597
epoch_num :  11
test_loss :  5.73350162851825
train_loss :  5.7431874400281115
epoch_num :  16
test_loss :  4.762742807632007
train_loss :  4.809705547499166
epoch_num :  21
test_loss :  4.109854460878398
train_loss :  4.107989584341822
epoch_num :  26
test_loss :  3.7546628587235364
train_loss :  3.6199230081892075
epoch_num :  31
test_loss :  3.5858790143381247
train_loss :  3.296036663466525
epoch_num :  36
test_loss :  3.484197953441186
train_loss :  3.07049353175832
epoch_num :  41
test_loss :  3.325560942753908
train_loss :  2.881667663789369
epoch_num :  46
test_loss :  3.141069593851535
train_loss :  2.7277442631092352
epoch_num :  51
test_loss :  2.9650614375501467
train_loss :  2.5996570461701616
epoch_num :  56
test_loss :  2.813461358157763
train_loss :  2.4

test_loss :  1.8628621295286516
train_loss :  1.585026607025363
epoch_num :  511
test_loss :  1.8572552908093984
train_loss :  1.5829281699666031
epoch_num :  516
test_loss :  1.8547615446618269
train_loss :  1.5810167638490136
epoch_num :  521
test_loss :  1.857469981365282
train_loss :  1.5791095651290858
epoch_num :  526
test_loss :  1.861307770880345
train_loss :  1.5780136368982762
epoch_num :  531
test_loss :  1.8645215492763942
train_loss :  1.5772917999155782
epoch_num :  536
test_loss :  1.8670235179706296
train_loss :  1.576838795592966
epoch_num :  541
test_loss :  1.868998341870171
train_loss :  1.5765956011288695
epoch_num :  546
test_loss :  1.8708626665650308
train_loss :  1.5766161642708725
epoch_num :  551
test_loss :  1.8732125576702443
train_loss :  1.5771063242724255
epoch_num :  556
test_loss :  1.8771786239160972
train_loss :  1.578446774679051
epoch_num :  561
test_loss :  1.884211516933094
train_loss :  1.5813286586403605
epoch_num :  566
test_loss :  1.88608397

In [ ]:
import matplotlib.pyplot as plt
# plt.subplot(211)
plt.plot(losses[1:], label = 'train_loss')
plt.plot(test_losses[1:], label = 'test_loss')
plt.legend()
plt.show()
# plt.subplot(212)
# plt.plot(np.multiply(pred_out(x_test, y_test, weights1, weights2, biases1, biases2)-y_test, pred_out(x_test, y_test, weights1, weights2, biases1, biases2)-y_test).T)

In [ ]:
ind = 101
print(pred_out(x_test, y_test, weights1, weights2, biases1, biases2)[:,5:10])
print(y_test[:,5:10])